In [24]:
from pathlib import Path
from lxml import etree

In [25]:
directory = 'data/dataset'
file_list = []
pathlist = Path(directory).glob('**/*.xml')
for path in pathlist:
    # because path is object not string
    file_list.append(str(path))

In [172]:
test_file = file_list[876]
with open(test_file) as fobj:
    xml = fobj.read()

In [173]:
root = etree.XML(bytes(bytearray(xml, encoding='utf-8')))

In [174]:
head = root[0]
candidate = head[0]
personnel = candidate.find('personnel')
text = head.find('text')
print('PROMPT KEY: ' + Path(test_file).parent.name)
print('SORT KEY: ' + head.get('sortkey'))
print('SCORE: ' + candidate.find('score').text)
print('LANGUAGE: ' + personnel.find('language').text)
print('AGE: ' + personnel.find('age').text)

PROMPT KEY: 0100_2000_6
SORT KEY: TR798*0100*2000*01
SCORE: 24.0
LANGUAGE: Chinese
AGE: 16-20


In [175]:
answers = []
for item in text.getchildren():
    print('TAG: ' + item.tag)
    print('QUESTION NUMBER: ' + item.find('question_number').text)
    print('EXAM SCORE: ' + item.find('exam_score').text)
    answers.append(item.find('coded_answer'))

TAG: answer1
QUESTION NUMBER: 1
EXAM SCORE: 3.2
TAG: answer2
QUESTION NUMBER: 2
EXAM SCORE: 2.3


In [190]:
p = answers[0][3]
ns = p.getchildren()
print(etree.tostring(p))

b'<p><NS type="MT"><i>The</i><c>Concerning the</c></NS> accommodation<NS type="MP"><c>,</c></NS> I would like to <NS type="RV"><i>live</i><c>stay</c></NS> in tents<NS type="RP"><i>. Because</i><c>, because</c></NS> I <NS type="TV"><i>never <NS type="RV"><i>live</i><c>stay</c></NS></i><c>have never stayed</c></NS> in <NS type="MD"><c>a</c></NS> <NS type="AGN"><i>tents</i><c>tent</c></NS> before. I think it <NS type="TV"><i>is</i><c>will be</c></NS> great and I want to try it.</p>\n          '


In [191]:
def build_sentence(data, corrections):
    res = data.text
    for errors in data.getchildren():
        # process the tag
        if (not errors.tail == ''):
            res = res + ' ' + errors.tail

def process_ns(data, corrections):
    correct, wrong = '', ''
    if (data.text is not None):
        correct = correct + data.text
        wrong = wrong + data.text
    tup_c, tup_i = '', ''
    
#     print('PROCESSING: ' + data.tag)
    
    for items in data.getchildren():
        if ((items.tag == 'i') or (items.tag == 'ns' and items.getparent().tag == 'i')):
#             print('INC')
            i_c, i_i = process_ns(items, corrections) #items.text
            if (i_i is not None):
                wrong = wrong + ' ' + i_i
                tup_i = i_i
        elif ((items.tag == 'c') or (items.tag == 'ns' and items.getparent().tag == 'c')):
#             print('COR')
            c_c, c_i = process_ns(items, corrections) #items.text
            if (c_c is not None):
                correct = correct + ' ' + c_c
                tup_c = c_c
        else:
            c, i = process_ns(items, corrections)
            if (c is not None):
                correct = correct + ' ' + c
            if (i is not None):
                wrong = wrong + ' ' + i
            
#         print(items.tag)
        if (items.tail is not None):
            correct = correct + ' ' + items.tail
            wrong = wrong + ' ' + items.tail
    if (data.tag == 'NS'):
        if (data.get('type') in corrections):
            corrections[data.get('type')].append({'i': tup_i, 'c': tup_c})
        else:
            corrections[data.get('type')] = [{'i': tup_i, 'c': tup_c}]
        
    return correct, wrong

In [192]:
print(etree.tostring(p))
cor = {}
c, i = process_ns(p, cor)
print(c)
print(i)
print(cor)

b'<p><NS type="MT"><i>The</i><c>Concerning the</c></NS> accommodation<NS type="MP"><c>,</c></NS> I would like to <NS type="RV"><i>live</i><c>stay</c></NS> in tents<NS type="RP"><i>. Because</i><c>, because</c></NS> I <NS type="TV"><i>never <NS type="RV"><i>live</i><c>stay</c></NS></i><c>have never stayed</c></NS> in <NS type="MD"><c>a</c></NS> <NS type="AGN"><i>tents</i><c>tent</c></NS> before. I think it <NS type="TV"><i>is</i><c>will be</c></NS> great and I want to try it.</p>\n          '
  Concerning the  accommodation  ,  I would like to   stay  in tents  , because  I   have never stayed  in   a    tent  before. I think it   will be  great and I want to try it.
  The  accommodation   I would like to   live  in tents  . Because  I   never   live  in      tents  before. I think it   is  great and I want to try it.
{'MT': [{'i': 'The', 'c': 'Concerning the'}], 'MP': [{'i': '', 'c': ','}], 'RV': [{'i': 'live', 'c': 'stay'}, {'i': 'live', 'c': 'stay'}], 'RP': [{'i': '. Because', 'c': '

In [171]:
# test_file2 = ''
# for i, c in enumerate(file_list):
#     if ('doc2.xml' in c):
#         test_file2 = c
#         print(i)
#         print(c)

876
data/dataset/0100_2000_6/doc2.xml
